# Telco Customer Churn - Exploratory Data Analysis

This notebook explores churn patterns in the Telco dataset, with a focus on data quality,
summary statistics, and churn differences across key customer segments.


## Goals
- Quantify overall churn and the target distribution
- Surface churn differences by service, contract, and payment categories
- Compare numeric drivers such as tenure and charges


## Setup

In [10]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

plt.rcParams.update({
    "figure.figsize": (10, 5),
    "axes.grid": True,
    "axes.titlesize": 12,
    "axes.labelsize": 10,
})

DATA_PATH = Path("../data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
OUT_DIR = Path("../outputs")
FIG_DIR = OUT_DIR / "figures"
TAB_DIR = OUT_DIR / "tables"
FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)


## Load data

In [ ]:
df = pd.read_csv(DATA_PATH)
print(f"Shape: {df.shape}")
display(df.head())
df.info()


## Data quality checks

In [ ]:
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

missing = df.isna().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print("Missing values:")
display(missing)

cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
cat_summary = pd.DataFrame({"column": cat_cols, "nunique": [df[c].nunique() for c in cat_cols]})
display(cat_summary.sort_values("nunique", ascending=False))


## Cleaning and standardization

In [ ]:
df_clean = df.copy()

if "TotalCharges" in df_clean.columns:
    df_clean["TotalCharges"] = pd.to_numeric(df_clean["TotalCharges"], errors="coerce")
    missing_tc = df_clean["TotalCharges"].isna().sum()
    print("Missing TotalCharges after conversion:", missing_tc)
    df_clean["TotalCharges"] = df_clean["TotalCharges"].fillna(0)

if "SeniorCitizen" in df_clean.columns:
    df_clean["SeniorCitizen"] = df_clean["SeniorCitizen"].map({0: "No", 1: "Yes"}).astype("category")

if "Churn" in df_clean.columns:
    df_clean["Churn"] = df_clean["Churn"].astype(str).str.strip().astype("category")

if "customerID" in df_clean.columns:
    df_clean = df_clean.drop(columns=["customerID"])

df_clean.info()


## Target distribution

In [ ]:
churn_counts = df_clean["Churn"].value_counts(dropna=False)
churn_rate = (churn_counts / churn_counts.sum()).rename("rate")
display(pd.concat([churn_counts.rename("count"), churn_rate], axis=1))

ax = churn_counts.plot(kind="bar")
ax.set_title("Churn count")
ax.set_xlabel("Churn")
ax.set_ylabel("Count")
plt.tight_layout()
plt.savefig(FIG_DIR / "churn_count.png", dpi=200)
plt.show()

print(f"Churn rate (Yes): {churn_rate.get('Yes', np.nan):.3f}")


## Univariate numeric distributions

In [ ]:
num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", num_cols)

def plot_hist(series, title, fname):
    s = series.dropna()
    plt.figure()
    plt.hist(s, bins=30)
    plt.title(title)
    plt.xlabel(series.name)
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(FIG_DIR / fname, dpi=200)
    plt.show()

def plot_box_by_churn(col, fname):
    yes = df_clean.loc[df_clean["Churn"] == "Yes", col].dropna()
    no = df_clean.loc[df_clean["Churn"] == "No", col].dropna()

    plt.figure()
    plt.boxplot([no, yes], labels=["No", "Yes"])
    plt.title(f"{col} by churn")
    plt.xlabel("Churn")
    plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(FIG_DIR / fname, dpi=200)
    plt.show()

for c in [x for x in ["tenure", "MonthlyCharges", "TotalCharges"] if x in df_clean.columns]:
    plot_hist(df_clean[c], f"Distribution of {c}", f"hist_{c}.png")
    plot_box_by_churn(c, f"box_{c}_by_churn.png")


## Churn rate by category

In [ ]:
def churn_rate_by_category(col, min_count=50):
    tmp = (
        df_clean.groupby(col)["Churn"]
        .value_counts(normalize=True)
        .rename("rate")
        .reset_index()
    )
    tmp_yes = tmp[tmp["Churn"] == "Yes"].drop(columns=["Churn"])

    counts = df_clean[col].value_counts().rename("count")
    out = tmp_yes.merge(counts, left_on=col, right_index=True)
    out = out[out["count"] >= min_count].sort_values("rate", ascending=False)
    return out

def plot_churn_rate_by_category(col, fname, min_count=50):
    rates = churn_rate_by_category(col, min_count=min_count)
    if rates.empty:
        print(f"Skipping {col}: no categories meet min_count={min_count}")
        return

    plt.figure(figsize=(10, 5))
    plt.bar(rates[col].astype(str), rates["rate"])
    plt.title(f"Churn rate by {col} (min_count={min_count})")
    plt.ylabel("Churn rate (Yes)")
    plt.xticks(rotation=30, ha="right")
    plt.tight_layout()
    plt.savefig(FIG_DIR / fname, dpi=200)
    plt.show()

priority_cats = [
    c for c in ["Contract", "PaymentMethod", "InternetService", "TechSupport", "OnlineSecurity"]
    if c in df_clean.columns
]
for c in priority_cats:
    display(churn_rate_by_category(c, min_count=50).head(20))
    plot_churn_rate_by_category(c, f"churn_rate_by_{c}.png", min_count=50)


## Statistical tests on numeric drivers

In [ ]:
def numeric_tests(col):
    yes = df_clean.loc[df_clean["Churn"] == "Yes", col].dropna()
    no = df_clean.loc[df_clean["Churn"] == "No", col].dropna()

    t_stat, t_p = stats.ttest_ind(yes, no, equal_var=False)
    u_stat, u_p = stats.mannwhitneyu(yes, no, alternative="two-sided")

    mean_yes, mean_no = yes.mean(), no.mean()
    sd_yes, sd_no = yes.std(ddof=1), no.std(ddof=1)
    n_yes, n_no = len(yes), len(no)
    pooled_sd = (
        np.sqrt(((n_yes - 1) * sd_yes ** 2 + (n_no - 1) * sd_no ** 2) / (n_yes + n_no - 2))
        if (n_yes + n_no - 2) > 0
        else np.nan
    )
    cohens_d = (mean_yes - mean_no) / pooled_sd if pooled_sd and pooled_sd > 0 else np.nan

    return pd.Series({
        "mean_churn_yes": mean_yes,
        "mean_churn_no": mean_no,
        "t_pvalue": t_p,
        "mw_pvalue": u_p,
        "cohens_d": cohens_d,
        "n_yes": n_yes,
        "n_no": n_no,
    })

num_test_cols = [c for c in ["tenure", "MonthlyCharges", "TotalCharges"] if c in df_clean.columns]
num_results = pd.DataFrame({c: numeric_tests(c) for c in num_test_cols}).T
display(num_results.sort_values("t_pvalue"))
num_results.to_csv(TAB_DIR / "numeric_tests.csv", index=True)


## Quick summary

In [ ]:
print(f"Overall churn rate (Yes): {churn_rate.get('Yes', np.nan):.3f}")

for c in priority_cats:
    top_rates = churn_rate_by_category(c, min_count=100).head(3)
    if top_rates.empty:
        continue
    print(f"Top churn segments for {c}:")
    display(top_rates)

display(df_clean.describe())
